# Lab 12: API Setup and Configuration

## Learning Objectives
- Configure FastAPI application foundation
- Set up database connection manager
- Define Pydantic models for data validation
- Establish environment configuration

**Duration:** 45 minutes  
**Prerequisites:** Completion of Lab 7 (Python Driver & Service Architecture)  
**Database State:** Starting with 650 nodes, 800 relationships

## Step 1: Development Environment Verification

This cell verifies that all required packages are installed and the development environment is properly configured.

In [ ]:
# Cell 1: Development environment verification
import subprocess
import sys
import os
from pathlib import Path

def verify_development_environment():
    """Comprehensive verification of development setup"""
    
    print("🔧 DEVELOPMENT ENVIRONMENT VERIFICATION")
    print("=" * 50)
    
    # 1. Python version check
    python_version = sys.version_info
    print(f"Python Version: {python_version.major}.{python_version.minor}.{python_version.micro}")
    
    if python_version >= (3, 8):
        print("✓ Python version meets requirements (3.8+)")
    else:
        print("✗ Python version too old. Please upgrade to 3.8+")
        return False
    
    # 2. Required packages verification
    required_packages = [
        "fastapi", "uvicorn", "neo4j", "pydantic", 
        "python_jose", "passlib", "pytest"
    ]
    
    missing_packages = []
    for package in required_packages:
        try:
            __import__(package.replace("-", "_"))
            print(f"✓ {package} is available")
        except ImportError:
            missing_packages.append(package)
            print(f"✗ {package} is missing")
    
    if missing_packages:
        print(f"\nInstalling missing packages: {', '.join(missing_packages)}")
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", 
            *missing_packages
        ])
    
    # 3. Jupyter Lab setup verification
    lab_dir = Path("lab_13")
    if not lab_dir.exists():
        print("Creating lab directory structure in Jupyter Lab...")
        lab_dir.mkdir(exist_ok=True)
        print("✓ Use Jupyter Lab file browser to navigate to lab_13/")
    else:
        print("✓ Lab directory structure exists in Jupyter Lab")
    
    # 4. Environment file verification  
    env_file = lab_dir / ".env"
    if not env_file.exists():
        print("⚠ Create .env file using Jupyter Lab text editor")
        print("  Click '+' → 'Text File' → Save as '.env'")
    else:
        print("✓ Environment file exists")
    
    # 5. Neo4j connection test
    try:
        from neo4j import GraphDatabase
        driver = GraphDatabase.driver(
            "bolt://localhost:7687", 
            auth=("neo4j", "password")
        )
        with driver.session() as session:
            result = session.run("RETURN 1 as test")
            record = result.single()
            if record and record["test"] == 1:
                print("✓ Neo4j connection successful")
            else:
                print("✗ Neo4j connection test failed")
        driver.close()
    except Exception as e:
        print(f"✗ Neo4j connection failed: {e}")
        return False
    
    print("\n🎯 DEVELOPMENT ENVIRONMENT READY")
    print("All prerequisites verified successfully!")
    return True

# Run verification
verify_development_environment()

## Step 2: FastAPI Application Foundation

Configure the core FastAPI application with CORS middleware and global configuration.

In [ ]:
# Cell 2: FastAPI application foundation
import os
from pathlib import Path
from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
from datetime import datetime, timedelta
import uvicorn

# Load environment variables
load_dotenv("lab_13/.env")

# FastAPI application configuration
app = FastAPI(
    title="Neo4j Insurance API",
    description="Production-ready insurance management API built with Neo4j and FastAPI",
    version="1.0.0",
    docs_url="/docs",
    redoc_url="/redoc",
    openapi_url="/openapi.json"
)

# CORS middleware configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "http://localhost:8080", "http://localhost:8000"],
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE", "PATCH"],
    allow_headers=["*"],
)

# Global configuration
CONFIG = {
    "neo4j_uri": os.getenv("NEO4J_URI", "bolt://localhost:7687"),
    "neo4j_username": os.getenv("NEO4J_USERNAME", "neo4j"),
    "neo4j_password": os.getenv("NEO4J_PASSWORD", "password"),
    "neo4j_database": os.getenv("NEO4J_DATABASE", "neo4j"),
    "secret_key": os.getenv("SECRET_KEY", "your-secret-key"),
    "algorithm": os.getenv("ALGORITHM", "HS256"),
    "access_token_expire_minutes": int(os.getenv("ACCESS_TOKEN_EXPIRE_MINUTES", "30")),
    "api_host": os.getenv("API_HOST", "0.0.0.0"),
    "api_port": int(os.getenv("API_PORT", "8000"))
}

print("✓ FastAPI application configured")
print(f"✓ Database URI: {CONFIG['neo4j_uri']}")
print(f"✓ API will run on {CONFIG['api_host']}:{CONFIG['api_port']}")

## Step 3: Database Connection Manager

Implement production-grade Neo4j connection manager with connection pooling, retry logic, and health checks.

In [ ]:
# Cell 3: Enhanced database connection manager for API
from neo4j import GraphDatabase
from contextlib import contextmanager
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class APIConnectionManager:
    """Production-grade Neo4j connection manager for API applications"""
    
    def __init__(self, uri: str, username: str, password: str, database: str):
        self.uri = uri
        self.username = username
        self.password = password
        self.database = database
        self.driver = None
        self._connect()
    
    def _connect(self):
        """Establish connection with retry logic"""
        try:
            self.driver = GraphDatabase.driver(
                self.uri,
                auth=(self.username, self.password),
                max_connection_lifetime=1800,  # 30 minutes
                max_connection_pool_size=50,
                connection_acquisition_timeout=60,
                encrypted=False
            )
            # Verify connectivity
            with self.driver.session(database=self.database) as session:
                session.run("RETURN 1").consume()
            logger.info("✓ Neo4j connection established successfully")
        except Exception as e:
            logger.error(f"✗ Failed to connect to Neo4j: {e}")
            raise
    
    @contextmanager
    def get_session(self):
        """Context manager for database sessions"""
        session = self.driver.session(database=self.database)
        try:
            yield session
        finally:
            session.close()
    
    def execute_query(self, query: str, parameters: Dict = None):
        """Execute a single query with error handling"""
        try:
            with self.get_session() as session:
                result = session.run(query, parameters or {})
                return [record.data() for record in result]
        except Exception as e:
            logger.error(f"Query execution failed: {e}")
            raise HTTPException(
                status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
                detail=f"Database query failed: {str(e)}"
            )
    
    def execute_write_query(self, query: str, parameters: Dict = None):
        """Execute write query with transaction handling"""
        try:
            with self.get_session() as session:
                result = session.write_transaction(
                    lambda tx: tx.run(query, parameters or {}).data()
                )
                return result
        except Exception as e:
            logger.error(f"Write query execution failed: {e}")
            raise HTTPException(
                status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
                detail=f"Database write operation failed: {str(e)}"
            )
    
    def health_check(self) -> Dict[str, Any]:
        """Comprehensive database health check"""
        try:
            with self.get_session() as session:
                # Connection test
                start_time = datetime.now()
                session.run("RETURN 1").consume()
                response_time = (datetime.now() - start_time).total_seconds() * 1000
                
                # Database statistics
                stats_result = session.run("""
                    MATCH (n)
                    WITH count(n) AS nodeCount, count(DISTINCT labels(n)) AS labelCount
                    MATCH ()-[r]->()
                    WITH nodeCount, labelCount, count(r) AS relCount, count(DISTINCT type(r)) AS relTypeCount
                    RETURN nodeCount, relCount, labelCount, relTypeCount
                """)
                stats = stats_result.single()
                
                return {
                    "status": "healthy",
                    "response_time_ms": round(response_time, 2),
                    "database": self.database,
                    "statistics": {
                        "total_nodes": stats["nodeCount"] if stats else 0,
                        "total_relationships": stats["relCount"] if stats else 0,
                        "label_count": stats["labelCount"] if stats else 0,
                        "relationship_types": stats["relTypeCount"] if stats else 0
                    }
                }
        except Exception as e:
            return {
                "status": "unhealthy",
                "error": str(e),
                "database": self.database
            }
    
    def close(self):
        """Close database connection"""
        if self.driver:
            self.driver.close()
            logger.info("✓ Database connection closed")

# Initialize connection manager
connection_manager = APIConnectionManager(
    uri=CONFIG["neo4j_uri"],
    username=CONFIG["neo4j_username"],
    password=CONFIG["neo4j_password"],
    database=CONFIG["neo4j_database"]
)

print("✓ API-specific database connection manager initialized")

## Step 4: Pydantic Models

Define comprehensive Pydantic models for data validation, serialization, and API request/response handling.

In [ ]:
# Cell 4: Comprehensive Pydantic models for API requests/responses
from pydantic import BaseModel, Field, EmailStr, validator
from typing import Optional, List, Dict, Any
from datetime import datetime, date
from enum import Enum

# Enum definitions
class PolicyStatus(str, Enum):
    ACTIVE = "Active"
    PENDING = "Pending"
    EXPIRED = "Expired"
    CANCELLED = "Cancelled"

class ClaimStatus(str, Enum):
    SUBMITTED = "Submitted"
    UNDER_REVIEW = "Under Review"
    APPROVED = "Approved"
    REJECTED = "Rejected"
    SETTLED = "Settled"

class UserRole(str, Enum):
    CUSTOMER = "customer"
    AGENT = "agent"
    ADJUSTER = "adjuster"
    ADMIN = "admin"

# Authentication models
class UserLogin(BaseModel):
    username: str = Field(..., min_length=3, max_length=50)
    password: str = Field(..., min_length=6)

class Token(BaseModel):
    access_token: str
    token_type: str = "bearer"
    expires_in: int
    user_id: str
    role: UserRole

class UserProfile(BaseModel):
    user_id: str
    username: str
    email: EmailStr
    full_name: str
    role: UserRole
    is_active: bool = True
    created_date: datetime

# Customer models
class CustomerCreate(BaseModel):
    first_name: str = Field(..., min_length=1, max_length=50)
    last_name: str = Field(..., min_length=1, max_length=50)
    email: EmailStr
    phone: str = Field(..., regex=r"^\+?1?\d{9,15}$")
    date_of_birth: date
    address: str = Field(..., min_length=10, max_length=200)
    city: str = Field(..., min_length=2, max_length=50)
    state: str = Field(..., min_length=2, max_length=50)
    zip_code: str = Field(..., regex=r"^\d{5}(-\d{4})?$")
    
    @validator('date_of_birth')
    def validate_age(cls, v):
        if v > date.today():
            raise ValueError('Date of birth cannot be in the future')
        age = (date.today() - v).days / 365.25
        if age < 18:
            raise ValueError('Customer must be at least 18 years old')
        return v

class CustomerUpdate(BaseModel):
    first_name: Optional[str] = None
    last_name: Optional[str] = None
    email: Optional[EmailStr] = None
    phone: Optional[str] = None
    address: Optional[str] = None
    city: Optional[str] = None
    state: Optional[str] = None
    zip_code: Optional[str] = None

class CustomerResponse(BaseModel):
    customer_id: str
    first_name: str
    last_name: str
    email: str
    phone: str
    date_of_birth: date
    address: str
    city: str
    state: str
    zip_code: str
    customer_since: datetime
    total_policies: int = 0
    total_premium: float = 0.0
    risk_score: Optional[float] = None

# Policy models
class PolicyCreate(BaseModel):
    customer_id: str
    product_name: str = Field(..., min_length=1, max_length=100)
    coverage_amount: float = Field(..., gt=0, le=10000000)
    premium_amount: float = Field(..., gt=0, le=100000)
    policy_term_months: int = Field(..., ge=1, le=360)
    deductible: float = Field(..., ge=0, le=50000)
    
    @validator('premium_amount', 'coverage_amount', 'deductible')
    def round_currency(cls, v):
        return round(v, 2)

class PolicyUpdate(BaseModel):
    coverage_amount: Optional[float] = None
    premium_amount: Optional[float] = None
    deductible: Optional[float] = None
    status: Optional[PolicyStatus] = None

class PolicyResponse(BaseModel):
    policy_id: str
    policy_number: str
    customer_id: str
    customer_name: str
    product_name: str
    status: PolicyStatus
    coverage_amount: float
    premium_amount: float
    deductible: float
    policy_term_months: int
    start_date: date
    end_date: date
    created_date: datetime

# Claims models
class ClaimCreate(BaseModel):
    policy_id: str
    incident_date: date
    claim_amount: float = Field(..., gt=0, le=10000000)
    description: str = Field(..., min_length=10, max_length=1000)
    incident_type: str = Field(..., min_length=1, max_length=50)
    location: str = Field(..., min_length=5, max_length=200)
    
    @validator('incident_date')
    def validate_incident_date(cls, v):
        if v > date.today():
            raise ValueError('Incident date cannot be in the future')
        if v < date.today() - timedelta(days=365):
            raise ValueError('Incident date cannot be more than 1 year ago')
        return v

class ClaimUpdate(BaseModel):
    claim_amount: Optional[float] = None
    description: Optional[str] = None
    status: Optional[ClaimStatus] = None

class ClaimResponse(BaseModel):
    claim_id: str
    claim_number: str
    policy_id: str
    policy_number: str
    customer_name: str
    status: ClaimStatus
    claim_amount: float
    incident_date: date
    filed_date: datetime
    description: str
    incident_type: str
    location: str
    adjuster_name: Optional[str] = None

# Analytics models
class CustomerAnalytics(BaseModel):
    total_customers: int
    new_customers_this_month: int
    average_customer_value: float
    top_customers_by_premium: List[Dict[str, Any]]

class PolicyAnalytics(BaseModel):
    total_policies: int
    active_policies: int
    total_premium_collected: float
    average_policy_value: float
    policies_by_status: Dict[str, int]

class ClaimAnalytics(BaseModel):
    total_claims: int
    pending_claims: int
    total_claim_amount: float
    average_claim_amount: float
    claims_by_status: Dict[str, int]

# Response wrapper models
class APIResponse(BaseModel):
    success: bool = True
    message: str = "Operation completed successfully"
    data: Optional[Any] = None
    errors: Optional[List[str]] = None

class PaginatedResponse(BaseModel):
    items: List[Any]
    total: int
    page: int = 1
    per_page: int = 10
    pages: int

print("✓ Comprehensive Pydantic models defined")
print("✓ Input validation and response formatting ready")